## <center>Introduction to Cross Validation</center>

In this lecture series we will do a much deeper dive into various methods of cross-validation. As well as a discussion on the general philosphy behind cross validation. A nice official documentation guide can be found here: https://scikit-learn.org/stable/modules/cross_validation.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("Advertising.csv")

In [3]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


### Train | Test Split Procedure 

1. Clean and adjust data as necessary for X and y
2. Split Data in Train/Test for both X and y
3. Fit/Train Scaler on Training X Data
4. Scale X Test Data
5. Create Model
6. Fit/Train Model on X Train Data
7. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
8. Adjust Parameters as Necessary and repeat steps 6 and 7

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [5]:
# Points 1-4

X = df.drop('sales',axis=1)
y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Points 5-6
model = Ridge(alpha=100)
model.fit(X_train,y_train)

Ridge(alpha=100)

In [7]:
# Point 7
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred)

7.34177578903413

In [8]:
# Point 8
model = Ridge(alpha=1)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred)

2.319021579428752

Much better! We could repeat this until satisfied with performance metrics. (We previously showed RidgeCV can do this for us, but the purpose of this lecture is to generalize the CV process for any model).

----
### Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

1. Clean and adjust data as necessary for X and y
2. Split Data in Train/Validation/Test for both X and y
3. Fit/Train Scaler on Training X Data
4. Scale X Eval Data
5. Create Model
6. Fit/Train Model on X Train Data
7. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
8. Adjust Parameters as Necessary and repeat steps 6 and 7
9. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [9]:
X = df.drop('sales',axis=1)
y = df['sales']

In [10]:
######################################################################
#### SPLIT TWICE! Here we create TRAIN | VALIDATION | TEST  #########
####################################################################


# 70% of data is training data, set aside other 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)


# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

In [12]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)
model.fit(X_train,y_train)

Ridge(alpha=100)

In [13]:
y_eval_pred = model.predict(X_eval)

In [14]:
# Evaluation
mean_squared_error(y_eval,y_eval_pred)

7.320101458823871

In [15]:
# Adjust Parameters and Re-evaluate
model = Ridge(alpha=1)
model.fit(X_train,y_train)

Ridge(alpha=1)

In [16]:
y_eval_pred = model.predict(X_eval)

# Another evaluation
mean_squared_error(y_eval,y_eval_pred)

2.383783075056986

In [17]:
# Final Evaluation (Can no longer edit parameters after this!)
y_final_test_pred = model.predict(X_test)
mean_squared_error(y_test,y_final_test_pred)

2.254260083800517

----
### Cross Validation with cross_val_score

<img src="grid_search_cross_validation.png">

In [18]:
X = df.drop('sales',axis=1)
y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
model = Ridge(alpha=100)

In [20]:
from sklearn.model_selection import cross_val_score

In [21]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html

scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [22]:
# Average of the MSE scores (we set back to positive)

abs(scores.mean())

8.215396464543607

In [23]:
# Adjust model based on metrics

model = Ridge(alpha=1)

In [24]:
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)
scores

array([-3.15513238, -1.58086982, -5.40455562, -2.21654481, -4.36709384])

In [25]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

3.344839296530696

In [26]:
# Final Evaluation (Can no longer edit parameters after this!)

# In this case, it won't be necessary to split the test set another time to get the validation set. 
# The same train test is divided in test and train sets with the cross validation.


# Need to fit the model first!
model.fit(X_train,y_train)

y_final_test_pred = model.predict(X_test)
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752

----
### Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

<ul>
<li>It allows specifying multiple metrics for evaluation.</li>
<li>It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.</li>
<li>For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
<ul>
<li>['test_score', 'fit_time', 'score_time']</li>
</ul></li>
<li>And for multiple metric evaluation, the return value is a dict with the following keys:<ul>
<li>['test_scorer1_name', 'test_scorer2_name', 'test_scorer...', 'fit_time', 'score_time']</li>
</ul> return_train_score is set to False by default to save computation time. To evaluate the scores on the training set as well you need to be set to True.</li>
</ul>

In [27]:
X = df.drop('sales',axis=1)
y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
model = Ridge(alpha=100)

In [29]:
from sklearn.model_selection import cross_validate

In [30]:
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

scores

{'fit_time': array([0.        , 0.00800252, 0.        , 0.        , 0.        ]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [31]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.000000,0.0,-2.312430,-9.325530,-6.449885
1,0.008003,0.0,-1.746534,-4.944962,-5.589261
2,0.000000,0.0,-2.562117,-11.396652,-10.339140
3,0.000000,0.0,-2.018732,-7.024211,-6.619504
4,0.000000,0.0,-2.279519,-8.385627,-7.755785


In [32]:
pd.DataFrame(scores).mean()

fit_time                        0.001601
score_time                      0.000000
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

In [33]:
model = Ridge(alpha=1)

In [34]:
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)
scores

{'fit_time': array([0., 0., 0., 0., 0.]),
 'score_time': array([0.00799584, 0.        , 0.        , 0.        , 0.        ]),
 'test_neg_mean_absolute_error': array([-1.54711694, -1.02604449, -1.40079299, -1.15425141, -1.47022164]),
 'test_neg_mean_squared_error': array([-3.15513238, -1.58086982, -5.40455562, -2.21654481, -4.36709384]),
 'test_max_error': array([-3.08829958, -2.81744088, -9.35320917, -4.05585583, -6.49092188])}

In [35]:
pd.DataFrame(scores).mean()

fit_time                        0.000000
score_time                      0.001599
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

In [36]:
model.fit(X_train,y_train)
y_final_test_pred = model.predict(X_test)
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752